In [1]:
##import packages here
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer

# We'll use this later
from numpy import meshgrid

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

## This sets the plot style
## to have a grid on a white background
sns.set_style("whitegrid")

In [2]:
sp500_df = pd.read_csv('sp500.csv', names = ['Date', 'Open', 'High', 'Low', 'Close'], header=0 )
sp500_df = sp500_df.dropna()
sp500_df['Date'] = pd.to_datetime(sp500_df['Date'])
sp500_df = sp500_df.set_index('Date')
sp500_df = sp500_df.drop(columns=['Open', 'High', 'Low'])
sp500_df.head()

,Close
Date,
2021-05-14,4173.85
2021-05-13,4112.50
2021-05-12,4063.04
2021-05-11,4152.10
2021-05-10,4188.43


In [3]:
tby_df = pd.read_csv('bond.csv', names = ['Date', 'Value'], header =0)
tby_df = tby_df.dropna()
tby_df['Date'] = pd.to_datetime(tby_df['Date'])
tby_df = tby_df.set_index('Date')
tby_df =tby_df.loc[tby_df.index[::-1]]
tby_df.head()

,Value
Date,
2021-05-06,1.58
2021-05-05,1.59
2021-05-04,1.61
2021-05-03,1.63
2021-04-30,1.65


In [4]:
ffr_df = pd.read_csv('ffr.csv', names = ['Date', 'FedFunds'], header=0)
ffr_df = ffr_df.dropna()
ffr_df['Date'] = pd.to_datetime(ffr_df['Date'])
ffr_df.set_index('Date', inplace=True)
ffr_df =ffr_df.loc[ffr_df.index[::-1]]
ffr_df.head()

,FedFunds
Date,
2021-04-01,0.07
2021-03-01,0.07
2021-02-01,0.08
2021-01-01,0.09
2020-12-01,0.09


In [5]:
fta_df = pd.read_csv('fedtotalassets.csv', names = ['Date', 'FedTotalAssets'], header=0)
fta_df = fta_df.dropna()
fta_df['Date'] = pd.to_datetime(fta_df['Date'])
fta_df.set_index('Date', inplace=True)
fta_df =fta_df.loc[fta_df.index[::-1]]
fta_df.head()

,FedTotalAssets
Date,
2021-05-12,7830663.0
2021-05-05,7810486.0
2021-04-28,7780962.0
2021-04-21,7820948.0
2021-04-14,7793104.0


In [6]:
eps_df =pd.read_csv('sp500eps.csv', names =['Date', 'Value'], header =0)
eps_df = eps_df.dropna()
eps_df['Date'] = pd.to_datetime(eps_df['Date'])
eps_df.set_index('Date', inplace=True)
eps_df.head()

,Value
Date,
2020-12-31,95.72
2020-11-30,97.20
2020-10-31,98.53
2020-09-30,99.95
2020-08-31,100.44


In [7]:
div_df = pd.read_csv('sp500div.csv', names=['Date', 'Value'], header=0)
div_df = div_df.dropna()
div_df['Date'] = pd.to_datetime(div_df['Date']) 
div_df.set_index('Date', inplace=True)
div_df.head()

,Value
Date,
2021-05-31,1.37
2021-04-30,1.41
2021-04-01,1.45
2021-03-31,1.47
2021-03-01,1.49


In [8]:
vix_df = pd.read_csv('VIX.csv')
vix_df = vix_df.rename(columns={'Trade Date': 'Date'})
vix_df['Date'] = pd.to_datetime(vix_df['Date'])
vix_df = vix_df.set_index('Date')
vix_df

,Open,High,Low,Close,Settle,Change,Total Volume,EFP,Prev. Day Open Interest
Date,,,,,,,,,
2021-05-19,21.60,25.35,21.60,24.30,25.2100,4.0662,2686.0,0.0,17537.0
2021-05-18,19.70,21.90,18.79,21.76,21.1438,1.2375,79787.0,0.0,18216.0
2021-05-17,19.25,21.35,18.95,19.65,19.9063,0.7751,88206.0,45.0,32516.0
2021-05-14,22.75,22.90,18.89,19.10,19.1312,-3.5620,107377.0,0.0,46726.0
2021-05-13,26.25,27.03,22.12,22.70,22.6932,-3.2793,135964.0,0.0,58534.0
...,...,...,...,...,...,...,...,...,...
2005-06-24,150.90,152.00,150.50,152.00,152.7000,1.5000,9.0,0.0,117.0
2005-06-23,149.80,150.50,149.10,150.50,151.2000,2.6000,61.0,0.0,121.0
2005-06-22,148.40,148.40,148.40,148.40,148.6000,-0.5000,10.0,0.0,60.0


In [9]:
# drop rows of df2 whose date are not in df1
# def drop_date(df1, df2):
#     for i in df2.index:
#         if i not in df1.index:
#             df2 = df2.drop(i)

In [11]:
import datetime as datetime
def last_day_of_month(d):
    next_month = d.replace(day=28)+datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def first_day_of_next_month(d):
    dt = last_day_of_month(d)
    return dt + datetime.timedelta(days=1)

def end_date_calc(start_date, str1):
    # starts the first day of month till the frist day of the next month
    if (str1=='ffr'):
        end_date = first_day_of_next_month(start_date.date())
    # updates weekly
    if (str1 == 'fta'):
        end_date = start_date + pd.Timedelta(days=7)
    # starts at the end of month till the end of the next month
    if (str1 == 'eps'):
        end_date = last_day_of_month(start_date.date()+datetime.timedelta(days=1))
    # if next index ends with 01, then starts at 01 till the end of month
    if (str1 == 'div'):
        end_date = last_day_of_month(start_date.date()+ datetime.timedelta(days=1))
    return end_date    

def convert_to_daily(df, df2, str1, str2 ):
    for i in df2.index: 
        start_date = i 
        
        if (start_date + pd.Timedelta(days=1) in  df2.index):
            continue
        
        end_date = end_date_calc(start_date, str1 )
        if end_date <= df.index[-1]:
            continue
        delta = datetime.timedelta(days=1)      
        while start_date < end_date:
            
            if start_date in df.index:
                df.loc[start_date, str1] = df2[str2].loc[i]  
        
            start_date += delta 

In [12]:
from dateutil.relativedelta import relativedelta
sp500_df_min = sp500_df.index[np.argmin(sp500_df.index)]
sp500_df_max = sp500_df.index[np.argmax(sp500_df.index)]

tby_df_min = tby_df.index[np.argmin(tby_df.index)]
tby_df_max = tby_df.index[np.argmax(tby_df.index)]

ffr_df_min = ffr_df.index[np.argmin(ffr_df.index)]
ffr_df_max = ffr_df.index[np.argmax(ffr_df.index)] + relativedelta(months=1)

fta_df_min = fta_df.index[np.argmin(fta_df.index)]
fta_df_max = fta_df.index[np.argmax(fta_df.index)] + pd.Timedelta(weeks=1)

eps_df_min = eps_df.index[np.argmin(eps_df.index)] 
eps_df_max = eps_df.index[np.argmax(eps_df.index)] + relativedelta(months=1)

div_df_min = div_df.index[np.argmin(div_df.index)]
div_df_max = div_df.index[np.argmax(div_df.index)] + relativedelta(months=1)

In [13]:
min_date = max(sp500_df_min, tby_df_min, ffr_df_min,fta_df_min ,eps_df_min , div_df_min)
max_date = min(sp500_df_max,tby_df_max,ffr_df_max,fta_df_max, eps_df_max,div_df_max )
print(min_date, max_date)

2002-12-18 00:00:00 2021-01-31 00:00:00


In [14]:
def date_range(df):
    return [x for x in df.index if x <= max_date and x >= min_date]


df = sp500_df.loc[date_range(sp500_df), ].copy()
df['tby'] = tby_df['Value'] 
df['ffr'] = ffr_df['FedFunds']
df['fta'] = fta_df['FedTotalAssets']
df['eps'] = eps_df['Value']
df['div'] = div_df['Value']
df['vix'] = vix_df['Close']

In [15]:
ffr_df2 = ffr_df.loc[date_range(ffr_df), ].copy()
fta_df2 = fta_df.loc[date_range(fta_df), ].copy()
eps_df2 = eps_df.loc[date_range(eps_df), ].copy()
div_df2 = div_df.loc[date_range(div_df), ].copy()

In [17]:
convert_to_daily(df, ffr_df2, 'ffr', 'FedFunds' )
convert_to_daily(df, fta_df2, 'fta', 'FedTotalAssets' )
convert_to_daily(df, eps_df2, 'eps', 'Value' )
convert_to_daily(df, div_df2, 'div', 'Value' )
df =df.loc[df.index[::-1]]
df = df.dropna()
df

,Close,tby,ffr,fta,eps,div,vix
Date,,,,,,,
2021-01-29,3714.24,1.11,0.09,7404926.0,95.72,1.58,32.85
2021-01-28,3787.38,1.07,0.09,7404926.0,95.72,1.58,30.49
2021-01-27,3750.77,1.04,0.09,7404926.0,95.72,1.58,33.05
2021-01-26,3849.62,1.05,0.09,7414942.0,95.72,1.58,25.30
2021-01-25,3855.36,1.05,0.09,7414942.0,95.72,1.58,25.85
...,...,...,...,...,...,...,...
2005-06-24,1191.57,3.92,3.04,811179.0,84.81,1.76,152.00
2005-06-23,1200.73,3.96,3.04,811179.0,84.81,1.76,150.50
2005-06-22,1213.88,3.95,3.04,811179.0,84.81,1.76,148.40


In [18]:
df.to_csv(r'/Users/yuqingdai/Documents/GitHub/Stock-Erdos/scratch work/Yuqing-Data-Merge/df5.csv')

In [13]:
# non-periodic
# def convert_to_daily(df, df2, str1, str2 ):
#     i = len(df2.index)-1
#     while i >0:
#         start_date = df2.index[i]

#         if (start_date + pd.Timedelta(days=1) in  df2.index):
#             continue
        
#         end_date = df2.index[i-1]
#         if end_date <= df.index[-1]:
#             continue
            
            
#         delta = datetime.timedelta(days=1)      
#         while start_date < end_date:
#             if start_date in df.index:
#                 df.loc[start_date, str1] = df2[str2].loc[df2.index[i]]  
        
#             start_date += delta 
        
#         i -= 1

# import datetime as datetime
# df = sp500_df.copy()
# drop_date(df,div_df)
# df['div'] = div_df['Value']

# convert_to_daily(df, div_df, 'div', 'Value' )
# df.dropna()
# df